# Geospatial analysis and visualization with PyGMT

Here, we'll demonstrate new features for data processing and visualization in PyGMT.

## Data visualization

Data visualization with PyGMT is handled through the [pygmt.Figure class and its methods](https://www.pygmt.org/latest/api/index.html#plotting). Examples include:

- Project and plot grids or images
- Create 3-D perspective plots
- Plot velocity vectors, crosses, and anisotropy bars
- Plot earthquake focal mechanisms


In [ ]:
import pygmt
import geopandas as gpd

### Plot earthquake focal mechanisms

First, we'll demonstrate how PyGMT provides access to the GMT's geophysical data visualization tools based on an example from the [UNAVCO-sponsored GMT for Geodesy course](https://github.com/GenericMappingTools/gmt-for-geodesy) using data from the [Global Centroid-Moment-Tensor Project](https://www.globalcmt.org/).

In [ ]:
# Load SRTM15+ earth relief data (Tozer et al., 2019)
grid = pygmt.datasets.load_earth_relief(
    resolution="02m", region=[122, 147, 30, 48], registration="gridline"
)
grid

In [ ]:
fig = pygmt.Figure()
# Plot earth relief data
fig.grdimage(
    grid=grid, shading="+a", projection="M15c", frame=["WSen", "af"], cmap="SCM/oleron"
)
# Add a colorbar for the earth relief data
fig.colorbar(position="jTR+o-2c/0c+w6c+ml", frame='a2000+l"Earth relief (m)"')
# Display the image
fig.show()

In [ ]:
# Create a colormap for the earthquake depth
pygmt.makecpt(cmap="plasma", series=[0, 700, 5], reverse=True)
# Plot earthquake focal mechanisms
fig.meca("_data/japan-focal.dat", convention="mt", scale="0.5c+f0p", C=True)
# Add a second colorbar for earthquake focal mechanisms
fig.colorbar(position="jBR+o-2c/0c+w-6c+ml", frame='a100+l"Focal depth (km)"')
# Display the image
fig.show()

### Plot geospatial vector data

One of the main benefits of PyGMT is the integration with the broader scientific Python ecosystem. In this section, we'll show how PyGMT can be used to visualize geopandas data objects based on one of the [PyGMT gallery examples](https://www.pygmt.org/latest/gallery/lines/roads.html#sphx-glr-gallery-lines-roads-py).


Read shapefile data using geopandas

In [ ]:
gdf = gpd.read_file(
    "http://www2.census.gov/geo/tiger/TIGER2015/PRISECROADS/tl_2015_15_prisecroads.zip"
)

Select a few road types to plot

In [ ]:
roads_common = gdf[gdf.RTTYP == "M"]  # Common name roads
roads_state = gdf[gdf.RTTYP == "S"]  # State recognized roads
roads_interstate = gdf[gdf.RTTYP == "I"]  # Interstate roads

In [ ]:
fig = pygmt.Figure()
# Define target region around O'ahu (Hawai'i)
region = [-158.3, -157.6, 21.2, 21.75]  # xmin, xmax, ymin, ymax
# Specify the figure title
title = r"Main roads of O\047ahu (Hawai\047i)"  # \047 is octal code for '
# Plot a basemap with a title
fig.basemap(region=region, projection="M12c", frame=["af", f'WSne+t"{title}"'])
# Plot coastlines, color water blue, and land gray
fig.coast(land="gray", water="dodgerblue4", shorelines="1p,black")
# Display the figure
fig.show()

In [ ]:
# Plot the individual road types with different pen settings and assign labels
# which are displayed in the legend
fig.plot(data=roads_common, pen="5p,dodgerblue", label="Common Name")
fig.plot(data=roads_state, pen="2p,gold", label="State Recognized")
fig.plot(data=roads_interstate, pen="2p,red", label="Interstate")
# Add legend
fig.legend()
# Display the figure
fig.show()

## Processing tables | grids

PyGMT's numerous functions for processing tabular or gridded data are detailed in the [API documentation](https://www.pygmt.org/latest/api/index.html#data-processing). Examples include:
- Grid arbitrarily spaced data
- Project data onto lines or great circles
- Clip, cut, fill, and sample grids
- Filter grids in the space of time domains

### Grid LiDAR data

In this example, we'll cover a simplified version of the 2022 EGU Short Course tutorial [LiDAR point clouds to 3D surfaces](https://www.generic-mapping-tools.org/egu22pygmt/lidar_to_surface.html)!

In [ ]:
import laspy
import pandas as pd
import pygmt

Download LiDAR LAZ file from a URL and load LAZ data into pandas DataFrame.

The data are from a recent LiDAR survey of Wellington, New Zealand that's published under a [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/) license.
- OpenTopography link: https://doi.org/10.5069/G9K935QX
- Bulk download location: https://opentopography.s3.sdsc.edu/minio/pc-bulk/NZ19_Wellington
- Official 1m DSM from LINZ: https://data.linz.govt.nz/layer/105024-wellington-city-lidar-1m-dsm-2019-2020

References:
- https://medium.com/on-location/from-points-to-pixels-creating-digital-elevation-models-from-open-topography-point-clouds-abe616d06860
- https://github.com/GenericMappingTools/foss4g2019oceania/blob/v1/3_lidar_to_surface.ipynb
- https://github.com/weiji14/30DayMapChallenge2021/blob/v0.3.1/day17_land.py

In [ ]:
lazfile = pygmt.which(
    fname="https://opentopography.s3.sdsc.edu/pc-bulk/NZ19_Wellington/CL2_BQ31_2019_1000_2138.laz",
    download=True,
)
# Load LAZ data into a pandas DataFrame
lazdata = laspy.read(source=lazfile)
df = pd.DataFrame(
    data={
        "x": lazdata.x.scaled_array(),
        "y": lazdata.y.scaled_array(),
        "z": lazdata.z.scaled_array(),
        "classification": lazdata.classification,
    }
)
df = df.query(expr="classification != 18")

Get bounding box region

In [ ]:
region = pygmt.info(data=df[["x", "y"]], spacing=1)

Preprocess LiDAR data using blockmedian

In [ ]:
df_trimmed = pygmt.blockmedian(
    data=df[["x", "y", "z"]],
    T=0.99,  # 99th quantile, i.e. the highest point
    spacing="1+e",
    region=region,
)
df_trimmed

Create a Digital Surface Elevation Model with a spatial resolution of 1m.

In [ ]:
grid = pygmt.surface(
    x=df_trimmed.x,
    y=df_trimmed.y,
    z=df_trimmed.z,
    spacing="1+e",
    region=region,  # xmin, xmax, ymin, ymax
    T=0.35,  # tension factor
)
grid

Plot the Digital Surface Model (DSM) by passing the
[xarray.DataArray](https://docs.xarray.dev/en/v2022.03.0/generated/xarray.DataArray.html)
grid into
[pygmt.Figure.grdimage](https://www.pygmt.org/v0.7.0/api/generated/pygmt.Figure.grdimage).

In [ ]:
fig = pygmt.Figure()
fig.grdview(
    grid=grid,
    cmap="bukavu",
    surftype="s",  # surface plot
    perspective=[315, 30],  # azimuth bearing, and elevation angle
    zscale=0.02,  # vertical exaggeration
    shading=True,  # hillshading
    region=region,
    frame=[
        "xaf+lEasting",
        "yaf+lNorthing",
        "zaf+lElevation (m)",
        "+tOriental Bay, Wellington",
    ],
)
fig.show()

### Perform grid histogram equalization

In this example, we'll explore a simplified version of the PyGMT tutorial on [transforming grids based on a cumulative distribution function](https://www.pygmt.org/latest/tutorials/advanced/grid_equalization.html#sphx-glr-tutorials-advanced-grid-equalization-py).

In [ ]:
region = [-119.825, -119.4, 37.6, 37.825]
grid = pygmt.datasets.load_earth_relief(
    resolution="03s", region=region, registration="gridline"
)
grid_dist = pygmt.grd2xyz(grid=grid, output_type="pandas")["elevation"]

In [ ]:
# Create an instance of the Figure class
fig = pygmt.Figure()
# Define figure configuration
pygmt.config(FORMAT_GEO_MAP="ddd.x", MAP_FRAME_TYPE="plain")
# Define the colormap for the figure
pygmt.makecpt(series=[500, 3540], cmap="turku")
# Setup subplots with two panels
with fig.subplot(
    nrows=1, ncols=2, figsize=("13.5c", "4c"), title="Digital Elevation Model"
):
    # Plot the original digital elevation model in the first panel
    with fig.set_panel(panel=0):
        fig.grdimage(grid=grid, region=region, projection="M?", frame="WSne", cmap=True)
    # Plot a histogram showing the z-value distribution in the original digital
    # elevation model
    with fig.set_panel(panel=1):
        fig.histogram(
            data=grid_dist,
            projection="X?",
            region=[500, 3600, 0, 20],
            series=[500, 3600, 100],
            frame=["wnSE", "xaf+lElevation (m)", "yaf+lPercent frequency"],
            cmap=True,
            histtype=1,
            pen="1p,black",
        )
        fig.colorbar(position="JMR+o1.5c/0c+w3c/0.3c", frame=True)
fig.show()

Create a new grid with the z-values representing the position of the original z-values in a given cumulative distribution.

In [ ]:
divisions = 9
linear = pygmt.grdhisteq.equalize_grid(grid=grid, divisions=divisions)
linear_dist = pygmt.grd2xyz(grid=linear, output_type="pandas")["z"]

Create a map showing the grid that has been transformed to have a linear distribution with nine divisions and a histogram of the data values.

In [ ]:
# Create an instance of the Figure class
fig = pygmt.Figure()
# Define figure configuration
pygmt.config(FORMAT_GEO_MAP="ddd.x", MAP_FRAME_TYPE="plain")
# Define the colormap for the figure
pygmt.makecpt(series=[0, divisions, 1], cmap="lajolla")
# Setup subplots with two panels
with fig.subplot(
    nrows=1, ncols=2, figsize=("13.5c", "4c"), title="Linear distribution"
):
    # Plot the grid with a linear distribution in the first panel
    with fig.set_panel(panel=0):
        fig.grdimage(
            grid=linear, region=region, projection="M?", frame="WSne", cmap=True
        )
    # Plot a histogram showing the linear z-value distribution
    with fig.set_panel(panel=1):
        fig.histogram(
            data=linear_dist,
            projection="X?",
            region=[-1, divisions, 0, 40],
            series=[0, divisions, 1],
            frame=["wnSE", "xaf+lDivision", "yaf+lPercent frequency"],
            cmap=True,
            histtype=1,
            pen="1p,black",
            center=True,
        )
        fig.colorbar(position="JMR+o1.5c/0c+w3c/0.3c", frame=True)
fig.show()

### Find out more

For more data visualization and processing examples, check out our [tutorials](https://www.pygmt.org/latest/tutorials/index.html), [gallery examples](https://www.pygmt.org/latest/gallery/index.html), and the [external resources](https://www.pygmt.org/latest/external_resources.html)!